# Customer Profile Clustering Analysis

This notebook performs K-Means clustering on customer data to identify distinct customer segments based on:
- Demographics (income, age, number of kids)
- Purchasing behavior (total spending, total purchases)
- Engagement (recency)

**Goal:** Identify actionable customer segments for targeted marketing campaigns.

## 1. Import Libraries

In [ ]:
import pandas as pd 
import psycopg2 #SQL Database Adapter for Python
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

## 2. Load Data from Database

Connect to PostgreSQL database and query customer features.

In [ ]:
connection = psycopg2.connect(
    host="localhost",
    database="marketing_db",
    user="postgres",
    password="Rc3288.2016"
)

query = '''
SELECT
    income,
    (2015 - year_birth) AS age,
    education,
    marital_status,
    (kidhome + teenhome) AS num_kids,
    (mntwines + mntfruits + mntmeatproducts + mntfishproducts + 
     mntsweetproducts + mntgoldprods) AS total_spending,
    (numdealspurchases + numwebpurchases + numcatalogpurchases + 
     numstorepurchases) AS total_purchases,
    recency
FROM marketing_data
WHERE income IS NOT NULL
;
'''

df = pd.read_sql(query, connection)
connection.close()

print(f"Loaded {len(df)} customer records")
df.head()

## 3. Data Exploration

Quick look at the data distribution and summary statistics.

In [ ]:
df.info()

In [ ]:
df.describe()

## 4. Feature Preparation

Select numerical features for clustering and handle missing values.

In [ ]:
features_for_clustering = df[['income', 'age', 'num_kids', 'total_spending', 'total_purchases', 'recency']].copy()
features_for_clustering = features_for_clustering.dropna()

print(f"Features shape: {features_for_clustering.shape}")
print(f"Missing values: {features_for_clustering.isnull().sum().sum()}")

## 5. Feature Scaling

Standardize features to have mean=0 and std=1 (important for K-Means as it uses distance metrics).

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_for_clustering)

print("Features scaled successfully")
print(f"Scaled features shape: {scaled_features.shape}")

## 6. Elbow Method - Finding Optimal K

Test different numbers of clusters to find the optimal value using the elbow method.

In [ ]:
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_features)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(10,6))
plt.plot(K_range, inertias, 'bo-')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method - Finding Optimal Clusters')
plt.grid(True)
plt.show()

## 7. Fit K-Means Model

Based on the elbow plot, fit the model with the optimal number of clusters.

In [ ]:
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df['cluster'] = kmeans.fit_predict(scaled_features)

print(f"Model fitted with {optimal_k} clusters")
print(f"\nCluster distribution:")
print(df['cluster'].value_counts().sort_index())

## 8. Cluster Profiles

Analyze the characteristics of each customer segment.

In [ ]:
cluster_profiles = df.groupby('cluster').agg({
    'income': 'mean',
    'age': 'mean',
    'num_kids': 'mean',
    'total_spending': 'mean',
    'total_purchases': 'mean',
    'recency': 'mean',
    'education': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown',
    'marital_status': lambda x: x.mode()[0] if not x.mode().empty else 'Unknown'
}).round(2)

cluster_profiles['count'] = df.groupby('cluster').size()

print("\n=== CUSTOMER SEGMENTS ===\n")
cluster_profiles

## 9. Detailed Cluster Analysis

Print detailed statistics for each cluster.

In [ ]:
for cluster_id in range(optimal_k):
    cluster_data = df[df['cluster'] == cluster_id]
    
    print(f"\n{'='*50}")
    print(f"CLUSTER {cluster_id}")
    print(f"{'='*50}")
    print(f"Size: {len(cluster_data)} customers ({len(cluster_data)/len(df)*100:.1f}%)")
    print(f"Average Income: ${cluster_data['income'].mean():,.0f}")
    print(f"Average Age: {cluster_data['age'].mean():.0f}")
    print(f"Average Kids: {cluster_data['num_kids'].mean():.1f}")
    print(f"Average Spending: ${cluster_data['total_spending'].mean():,.0f}")
    print(f"Average Purchases: {cluster_data['total_purchases'].mean():.1f}")
    print(f"Average Recency: {cluster_data['recency'].mean():.0f} days")
    print(f"Most Common Education: {cluster_data['education'].mode()[0]}")
    print(f"Most Common Marital Status: {cluster_data['marital_status'].mode()[0]}")

## 10. Visualize Customer Segments

Plot income vs. spending colored by cluster assignment.

In [ ]:
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df['income'], df['total_spending'], 
                     c=df['cluster'], cmap='viridis', alpha=0.6, s=50)
plt.xlabel('Income ($)', fontsize=12)
plt.ylabel('Total Spending ($)', fontsize=12)
plt.title('Customer Segments: Income vs. Total Spending', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.show()

## 11. Additional Visualizations

Create more plots to understand cluster characteristics.

In [ ]:
# Box plots for key metrics by cluster
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Cluster Characteristics Comparison', fontsize=16, fontweight='bold')

metrics = ['income', 'age', 'num_kids', 'total_spending', 'total_purchases', 'recency']
titles = ['Income', 'Age', 'Number of Kids', 'Total Spending', 'Total Purchases', 'Recency (days)']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    row = idx // 3
    col = idx % 3
    df.boxplot(column=metric, by='cluster', ax=axes[row, col])
    axes[row, col].set_title(title)
    axes[row, col].set_xlabel('Cluster')
    axes[row, col].set_ylabel(title)

plt.tight_layout()
plt.show()

## 12. Summary & Recommendations

**Key Findings:**
- The analysis identified 4 distinct customer segments
- Each segment has unique demographic and behavioral characteristics

**Next Steps:**
1. Name each cluster based on its characteristics (e.g., "High-Value Shoppers", "Budget-Conscious Families")
2. Develop targeted marketing strategies for each segment
3. Track segment migration over time
4. A/B test different approaches for each segment